In [1]:
import hashlib
import numpy as np
import os
#Thư viện os cung cấp các chức năng để tương tác với hệ điều hành
#Nó cho phép thực hiện các thao tác như đọc, ghi tệp, thay đổi thư mục làm việc, và quản lý quy trình
import pandas as pd
import multiprocessing
#hỗ trợ việc thực hiện các tác vụ song song bằng cách sử dụng nhiều tiến trình
#Điều này rất hữu ích để tăng hiệu suất khi xử lý các tác vụ nặng về tính toán.
from PIL import Image
#PIL cung cấp các công cụ để mở, thao tác và lưu các tệp ảnh
#Nó hỗ trợ nhiều định dạng ảnh và cung cấp các phương pháp để thực hiện các phép biến đổi cơ bản trên ảnh.
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#cho phép tạo ra các lô dữ liệu hình ảnh với các phép biến đổi ảnh (data augmentation) như xoay, dịch chuyển, cắt, và thay đổi độ sáng
#Điều này giúp tăng cường tính đa dạng của dữ liệu huấn luyện và giảm hiện tượng overfitting.
from keras.optimizers import Adam
#Adam là một thuật toán tối ưu hóa dựa trên gradient descent, thường được sử dụng để huấn luyện các mạng nơ-ron sâu
#Nó kết hợp các ưu điểm của hai phương pháp AdaGrad và RMSProp, giúp mô hình hội tụ nhanh và hiệu quả hơn.
import tensorflow as tf
#tf hỗ trợ tính toán bằng cách sử dụng GPU và TPU, và có thể được sử dụng cho cả nghiên cứu và ứng dụng sản phẩm.
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers, callbacks
#Thư viện layers của Keras cung cấp các lớp xây dựng mạng nơ-ron (Dense,...). Những lớp này giúp định nghĩa kiến trúc của mạng nơ-ron một cách trực quan.
#Thư viện models của Keras cung cấp các công cụ để định nghĩa và huấn luyện các mô hình mạng nơ-ron (Sequential,...)
#Thư viện optimizers của Keras cung cấp các thuật toán tối ưu hóa như SGD, Adam, RMSprop, giúp điều chỉnh các tham số của mô hình để giảm thiểu hàm mất mát.
#Thư viện callbacks của Keras cho phép bạn định nghĩa các hành động sẽ được thực hiện tại các điểm khác nhau trong quá trình huấn luyện (ví dụ như lưu trữ mô hình, giảm tốc độ học tập khi hiệu suất không cải thiện, dừng sớm khi mô hình không tiến bộ).

2024-07-05 11:56:13.736664: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 11:56:13.736763: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 11:56:13.868394: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
SPLITS_ID = 2

In [3]:
DATASET_PATH = "/kaggle/input/motocycledataset/Data"

In [4]:
BATCH_SIZE = 64
#"Batch size" là số lượng mẫu dữ liệu được đưa vào mô hình trong một lần huấn luyện (một batch)
WIDTH = 224
HEIGHT = 224
SEED = 42
#SEED khởi tạo bộ sinh số ngẫu nhiên
#Khi sử dụng cùng một seed, bạn sẽ có được cùng một chuỗi số ngẫu nhiên mỗi lần chạy, giúp tái hiện được kết quả.

# DATA LOADING

In [5]:
#Tạo đường dẫn đến file train và test theo SPLITS_ID
train_csv = os.path.join(DATASET_PATH, f"MotocycleDataset-Splits-{SPLITS_ID}-Train.csv")
test_csv = os.path.join(DATASET_PATH, f"MotocycleDataset-Splits-{SPLITS_ID}-Test.csv")

#Đọc file
#Tham số: header=None --> chỉ ra rằng tệp CSV không có dòng tiêu đề.
#Tham số: names=["file_path", "class"] --> gán tên cột đầu tiên là file_path và tên cột thứ hai là class.
train_df = pd.read_csv(train_csv, header=None, names=["file_path", "class"])
test_df = pd.read_csv(test_csv, header=None, names=["file_path", "class"])

#Cập nhật cột file_path trong DataFrame train_df, test_df bằng cách thêm đường dẫn cơ sở (DATASET_PATH) vào mỗi giá trị trong cột này
train_df["file_path"] = train_df["file_path"].apply(lambda x: os.path.join(DATASET_PATH, x))
test_df["file_path"] = test_df["file_path"].apply(lambda x: os.path.join(DATASET_PATH, x))

#Chuyển kiểu dữ liệua các gía trị trong cột class sang dạng string vì flow_from_dataframe cần đầu vào là kiểu string
train_df["class"] = train_df["class"].astype(str)
test_df["class"] = test_df["class"].astype(str)

# PREPROCESSING

In [6]:
image_set = set()
#Tạo 1 set dùng để lưu trữ các giá trị hash của các mảng numpy đại diện cho ảnh. Điều này giúp phát hiện các ảnh trùng lặp.

def hash_numpy_array(arr):
    '''
    Mục đích: Tạo giá trị hash từ một mảng numpy để kiểm tra trùng lặp ảnh.
    '''
    arr_bytes = arr.tobytes()
    #Chuyển đổi mảng numpy thành chuỗi bytes.
    
    hash_obj = hashlib.sha256(arr_bytes)
    #Tạo đối tượng hash SHA-256 từ chuỗi bytes.
    
    hash_hex = hash_obj.hexdigest()
    #Chuyển đổi đối tượng hash thành chuỗi hex.
    
    return hash_hex
    #Trả về chuỗi hex của giá trị hash.

def validate_image(image_path, skip_duplicate):
    '''
    Mục đích: Xác thực một ảnh.
    Cách hoạt động:
        - Kiểm tra xem đường dẫn ảnh có tồn tại và là một tệp tin không.
        - Mở ảnh và thay đổi kích thước.
        - Nếu skip_duplicate là True, tính toán giá trị hash của ảnh và kiểm tra xem giá trị này đã có trong image_set chưa.
            + Nếu đã có, trả về False.
            + Nếu chưa, thêm giá trị hash vào image_set.
        - Trả về True nếu ảnh hợp lệ.
    '''
    if not os.path.exists(image_path):
        return False
    if not os.path.isfile(image_path):
        return False
    try:
        with Image.open(image_path) as img:
            img.resize((WIDTH, HEIGHT))
            if skip_duplicate:
                hash_value = hash_numpy_array(np.array(img))
                if hash_value in image_set:
                    return False
                else:
                    image_set.add(hash_value)
        return True

    except Exception as e:
        print(e)
        return False
        
def validate_images_multicore(df, num_processes, skip_duplicate=False):
    '''
    Mục đích: Xác thực các ảnh trong DataFrame bằng cách sử dụng nhiều tiến trình.
    Cách hoạt động:
        - multiprocessing.Pool(num_processes): Tạo một pool với số lượng tiến trình được chỉ định bởi num_processes.
        - pool.starmap(validate_image, 
                       zip(df["file_path"], 
                       [skip_duplicate] * len(df))
                       ): Phân chia công việc xác thực ảnh giữa các tiến trình.
        - zip(df["file_path"],
              [skip_duplicate] * len(df)
              ): Kết hợp từng đường dẫn ảnh với giá trị skip_duplicate để truyền vào hàm validate_image.
        - return df[results]: Trả về DataFrame chỉ chứa các hàng mà ảnh đã được xác thực hợp lệ.
    '''
    with multiprocessing.Pool(num_processes) as pool:
        results = pool.starmap(
            validate_image, 
            zip(df["file_path"], [skip_duplicate] * len(df))
        )
    return df[results]  

In [7]:
train_df = validate_images_multicore(train_df, num_processes=16, skip_duplicate=True)
#sử dụng hàm validate_images_multicore để kiểm tra và xác thực các tệp ảnh trong DataFrame train_df bằng cách sử dụng 16 tiến trình.

#train_df là DataFrame chứa thông tin về các tệp ảnh cần xác thực

#num_processes chỉ định số lượng tiến trình sẽ được sử dụng trong pool
#Trong trường hợp này, 16 tiến trình sẽ được sử dụng
#Sử dụng nhiều tiến trình giúp phân chia công việc và xử lý nhiều tệp ảnh đồng thời, tăng tốc độ xử lý.

#Tham số skip_duplicate chỉ định liệu có bỏ qua các ảnh trùng lặp hay không
#Trong trường hợp này, skip_duplicate được đặt là False, nghĩa là hàm sẽ không kiểm tra và bỏ qua các ảnh trùng lặp.

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


image file is truncated (8 bytes not processed)
cannot identify image file '/kaggle/input/motocycledataset/Data/VinFast/22520968-22520996-22520999-22520929-22521373.VinFast.277.jpg'
cannot identify image file '/kaggle/input/motocycledataset/Data/VinFast/22520968-22520996-22520999-22520929-22521373.VinFast.311.jpg'
cannot identify image file '/kaggle/input/motocycledataset/Data/VinFast/22520968-22520996-22520999-22520929-22521373.VinFast.323.jpg'
cannot identify image file '/kaggle/input/motocycledataset/Data/Others/22520968-22520996-22520999-22520929-22521373.Others.567.jpg'
cannot identify image file '/kaggle/input/motocycledataset/Data/Others/22520968-22520996-22520999-22520929-22521373.Others.568.jpg'


In [8]:
image_set = set()
#Gán biến image_set bằng một tập hợp mới, chuẩn bị cho việc kiểm tra ảnh trùng trong tập test_df

In [9]:
test_df = validate_images_multicore(test_df, num_processes=16)
#Tham số skip_duplicate có giá trị mặc định là False.
#Điều này có nghĩa là hàm validate_image sẽ không thực hiện việc kiểm tra và bỏ qua các ảnh trùng lặp
#Do đó, tất cả các ảnh trong test_df sẽ được xử lý mà không quan tâm đến việc kiểm tra tính duy nhất của chúng dựa trên giá trị hash.

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


cannot identify image file '/kaggle/input/motocycledataset/Data/VinFast/22520968-22520996-22520999-22520929-22521373.VinFast.313.jpg'


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


# TRAINING

In [10]:
train_data_generator = ImageDataGenerator(
    rescale=1/255,
    validation_split=0.2,
)
#ImageDataGenerator:
#Tham số: rescale=1/255
    #Ban đầu: Giá trị của mỗi pixel trong một ảnh thường nằm trong khoảng [0, 255]
    #Sau khi chuẩn hóa: Bằng cách chia giá trị của mỗi pixel cho 255, ta thu được giá trị mới nằm trong khoảng [0, 1]
    #Mục đích chuẩn hóa: Thuận tiện tính toán, tăng tốc độ học.
#Tham số: validation_split=0.2
    #20% dữ liệu sẽ được sử dụng cho validation set, còn lại sẽ là training set.
    
test_data_generator = ImageDataGenerator(rescale=1/255)

dataframe_config = {
    'x_col': 'file_path', 
    'y_col': 'class',
    'target_size': (HEIGHT, WIDTH),
    'batch_size': BATCH_SIZE,
    'class_mode': 'categorical',
    'shuffle': True,
    'seed': SEED,
    'color_mode': 'rgb',
}
#x_col và y_col: Chỉ định tên cột trong DataFrame train_df và test_df lần lượt là "file_path" và "class"
#Cột "file_path" chứa đường dẫn tới các tệp ảnh, "class" chứa nhãn lớp của từng ảnh.

#target_size: Kích thước mà ảnh sẽ được chuyển đổi thành trước khi được đưa vào mô hình (HEIGHT, WIDTH).

#batch_size: Số lượng ảnh được tạo thành một batch để đưa vào mô hình trong mỗi lần huấn luyện.

#class_mode='categorical': Loại dữ liệu nhãn, ở đây là các nhãn được mã hóa one-hot vector.

#shuffle=True: Xáo trộn dữ liệu trong quá trình huấn luyện để đảm bảo mô hình học được tốt hơn.

#seed: Đặt seed để đảm bảo tính nhất quán của việc xáo trộn dữ liệu.

#color_mode='rgb': Chế độ màu của ảnh, ở đây là ảnh màu RGB.

train_generator = train_data_generator.flow_from_dataframe(train_df, **dataframe_config, subset='training')
val_generator = train_data_generator.flow_from_dataframe(train_df, **dataframe_config, subset='validation')
test_generator = train_data_generator.flow_from_dataframe(test_df, **dataframe_config)

#flow_from_dataframe: Phương thức này tạo ra một đối tượng dữ liệu sinh ra các batch từ DataFrame, thích hợp để sử dụng trong mô hình học sâu.
#subset='training' và subset='validation': Chỉ định rằng train_generator và val_generator chỉ nên sinh ra dữ liệu từ phần huấn luyện và validation của train_df, được xác định bởi validation_split trong train_data_generator.

Found 21459 validated image filenames belonging to 5 classes.
Found 5364 validated image filenames belonging to 5 classes.
Found 6916 validated image filenames belonging to 5 classes.


In [11]:
base_model = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

model = models.Sequential()
model.add(base_model)

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [12]:
learning_rate = 0.0001
epochs = 15


In [13]:
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [14]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator
)

#train_generator: Đối số đầu tiên train_generator cung cấp dữ liệu huấn luyện cho mô hình.

#epochs=epochs: Đối số epochs chỉ định số lượng lượt huấn luyện (epoch) mà mô hình sẽ được huấn luyện trên train_generator
#Mỗi epoch tương đương với việc đưa toàn bộ dữ liệu qua mô hình một lần.

#validation_data=val_generator: Đối số validation_data cho phép cung cấp dữ liệu validation để đánh giá hiệu suất của mô hình sau mỗi epoch.

Epoch 1/15


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
I0000 00:00:1720181061.420647     184 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1720181061.513939     184 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


208/336 ━━━━━━━━━━━━━━━━━━━━ 1:48 849ms/step - accuracy: 0.3980 - loss: 1.3846

W0000 00:00:1720181237.596402     184 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


336/336 ━━━━━━━━━━━━━━━━━━━━ 0s 766ms/step - accuracy: 0.4493 - loss: 1.2839

W0000 00:00:1720181325.094198     184 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


336/336 ━━━━━━━━━━━━━━━━━━━━ 424s 982ms/step - accuracy: 0.4496 - loss: 1.2832 - val_accuracy: 0.0000e+00 - val_loss: 2.6959
Epoch 2/15


W0000 00:00:1720181391.009550     182 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


336/336 ━━━━━━━━━━━━━━━━━━━━ 281s 820ms/step - accuracy: 0.7770 - loss: 0.6051 - val_accuracy: 0.0362 - val_loss: 3.6871
Epoch 3/15
336/336 ━━━━━━━━━━━━━━━━━━━━ 283s 828ms/step - accuracy: 0.8938 - loss: 0.3187 - val_accuracy: 0.3693 - val_loss: 2.2623
Epoch 4/15
336/336 ━━━━━━━━━━━━━━━━━━━━ 280s 819ms/step - accuracy: 0.9392 - loss: 0.1925 - val_accuracy: 0.5194 - val_loss: 2.5083
Epoch 5/15
336/336 ━━━━━━━━━━━━━━━━━━━━ 282s 824ms/step - accuracy: 0.9611 - loss: 0.1259 - val_accuracy: 0.4193 - val_loss: 2.9346
Epoch 6/15
336/336 ━━━━━━━━━━━━━━━━━━━━ 276s 808ms/step - accuracy: 0.9712 - loss: 0.0974 - val_accuracy: 0.5468 - val_loss: 2.3131
Epoch 7/15
336/336 ━━━━━━━━━━━━━━━━━━━━ 283s 828ms/step - accuracy: 0.9754 - loss: 0.0841 - val_accuracy: 0.3686 - val_loss: 3.5005
Epoch 8/15
336/336 ━━━━━━━━━━━━━━━━━━━━ 278s 810ms/step - accuracy: 0.9794 - loss: 0.0658 - val_accuracy: 0.4334 - val_loss: 3.3368
Epoch 9/15
336/336 ━━━━━━━━━━━━━━━━━━━━ 277s 812ms/step - accuracy: 0.9809 - loss: 0.06

# TESTING

In [15]:
loss, accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {accuracy:.6f}")

#model.evaluate(test_generator, steps=len(test_generator)): Đoạn này thực hiện việc đánh giá mô hình (model) bằng cách sử dụng dữ liệu từ test_generator
#Đối số steps=len(test_generator) cho biết số lượng bước (batch) mà generator sẽ tạo ra để đánh giá
#Mỗi bước sẽ tạo ra một batch dữ liệu để đưa vào mô hình để tính toán loss và accuracy.

109/109 ━━━━━━━━━━━━━━━━━━━━ 75s 691ms/step - accuracy: 0.6838 - loss: 1.9316
Test Accuracy: 0.685656


W0000 00:00:1720185426.923600     184 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


----------------------------------------------------

In [16]:
# test_df = validate_images_multicore(test_df, num_processes=12)

In [17]:
# data_generator = ImageDataGenerator(rescale=1/255)
# dataframe_config = {
#     'dataframe': test_df,
#     'x_col': 'file_path',
#     'y_col': 'class',
#     'target_size': (HEIGHT, WIDTH),
#     'batch_size': BATCH_SIZE,
#     'class_mode': 'categorical',
#     'shuffle': True,
#     'seed': SEED,
#     'color_mode': 'rgb',
# }
# test_generator = data_generator.flow_from_dataframe(**dataframe_config)

In [18]:
# loss, accuracy = model.evaluate(test_generator, steps=len(test_generator))
# print(f"Test Accuracy: {accuracy:.2f}")